In [2]:
import cv2 as cv
import numpy as np

import vision

In [3]:
# Vision example on an image

# load the image
img = cv.imread("testing_vision.jpg")
img = cv.resize(img, (640,480), interpolation=cv.INTER_CUBIC) 


# do the actual vision
original_coordinates, new_coordinates = vision.get_fop_coordinates(img)
fop = vision.get_fop(img, original_coordinates, new_coordinates)
obstacles = vision.get_obstacles(fop)
start_x, start_y, alpha = vision.get_starting_position(fop)
objective_x, objective_y = vision.get_objective(fop)

# draws everything and displays it
output = fop.copy()
cv.circle(output, (objective_x, objective_y), radius=10, color=(255, 0, 0), thickness=-1)
cv.circle(output, (start_x, start_y), radius=10, color=(0, 0, 255), thickness=-1)
for obstacle in obstacles:
    cv.drawContours(output, obstacle, -1, (0, 255, 0), 3)

print("Angle:" + str(alpha))

cv.imshow("image", output)

# press any key to close all windows
cv.waitKey(0) 
cv.destroyAllWindows()

Angle:[2.65711472]


In [4]:
print("Dimensions of the rectangular frame:")
print(new_coordinates[3][0], new_coordinates[3][1])

print("Starting coordinates and angle:") # angle of the robot relative to the x axis, counterclockwise, expressed in radian in range (-pi, pi]
print("(x ,y, alpha) = " + str(start_x) + ", "+ str(start_y) + ", " + str(alpha))

print("Vertices of the obstacles:")
print(obstacles)

print("Goal coordinates:")
print(objective_x, objective_y)


Dimensions of the rectangular frame:
580.0 369.0
Starting coordinates and angle:
(x ,y, alpha) = 533, 48, [2.65711472]
Vertices of the obstacles:
[[array([[[393, 166]],

       [[485, 163]],

       [[562, 312]],

       [[468, 352]],

       [[461, 293]],

       [[374, 254]]], dtype=int32)], [array([[[122,  26]],

       [[266, 142]],

       [[189, 248]],

       [[ 51, 126]],

       [[ 49, 104]]], dtype=int32)]]
Goal coordinates:
42 324


In [17]:
# 0 is laptop webcam, 1 is USB camera (!COMPUTER DEPENDENT!)
cap = cv.VideoCapture(0)

i = 0

# if unable to connect to the camera
if not (cap.isOpened()):
    print("Could not open video device")
else:
    while(True):
        # Get the input frame by frame (shape (480,640,3))
        ret, frame = cap.read() 

        # throws away the first 2 seconds of footage to let the camera warm up and focus
        i += 1
        if i < 60:
            continue
        
        # initialize vision: starting, obstacles and goal coordinates
        if i == 60:
            original_coordinates, new_coordinates = vision.get_fop_coordinates(frame)
            fop = vision.get_fop(frame, original_coordinates, new_coordinates)

            output = fop.copy()

            obstacles = vision.get_obstacles(fop)

            start_x, start_y, alpha = vision.get_starting_position(fop)

            objective_x, objective_y = vision.get_objective(fop)
            

        ################################## CALL FUNCTIONS HERE ################################## 
        
        fop = vision.get_fop(frame, original_coordinates, new_coordinates)
        center_x, center_y = vision.get_robot_center(fop)
        fop = cv.circle(fop, (center_x,center_y), radius=10, color=(0, 255, 0), thickness=-1)

        #########################################################################################
        
        cv.imshow('preview',fop)
        
        # Close all windows and get out of the loop if ESC is pressed
        if cv.waitKey(1) == 27:
            cv.destroyAllWindows()
            break